# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [132]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import string
string.punctuation

from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor



[nltk_data] Error loading punkt: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>
[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>
[nltk_data] Error loading wordnet: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>


In [133]:
# load data from database
engine = create_engine('sqlite:///UdacityDisasterResponse.db')
df = pd.read_sql_table("Messages_Categories", con=engine)

# df = df[:200]

# df = pd.read_sql_table("Messages_Categories", con="sqlite:///DisasterResponse.db")
X = df["message"]
Y = df.iloc[:,4:].astype("int")

df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [134]:
def tokenize(text):
    messages_tokenized = []
    
    for index, message in enumerate(text):
        message = message.lower()
        
        # Remove punctuation
        # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
        message = message.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenize into sentences and then words
        sentence_list = sent_tokenize(message)
        
        try:
            words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
        except:
            words_tokenized = message
#             print(Exception)
            print(f"Message {message} at index {index} caused an error. Message not tokenzied.")
            

        # Remove stopwords
        words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
        
        messages_tokenized.append(words_tokenized)
        
    return messages_tokenized


In [135]:
def tokenize_single(text):
    """
    Tokenize text into sentences and then single words. Stopwords are also removed.
    """
    
    message = text.lower()
    
    # Remove punctuation
    # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    message = message.translate(str.maketrans('', '', string.punctuation))
        
    # Tokenize into sentences and then words
    sentence_list = sent_tokenize(message)
        
    try:
        words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
    except:
        words_tokenized = [""]
        print(Exception)
        print(f"Text '{text}' replaced with empty string.")

    # Remove stopwords
    words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
    
    
    return words_tokenized


In [136]:
def lemmatize(text):
    """
    This function takes a series of text pieces, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per message.
    """
    # tokenize text
    messages_tokenized = tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    messages_lemmatized = []
    
    for message in messages_tokenized:
        try:
            words_lemmatized = [lemmatizer.lemmatize(token) for token in message]
            messages_lemmatized.append(words_lemmatized)
            max_index = messages_tokenized.index(message)
        except:
            print(f"Message at index {max_index + 1} caused an error. Message not lemmatized.")
            

    return messages_lemmatized

In [137]:
def lemmatize_single(text):
    """
    This function takes a piece of text, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per corpus.
    """
    # tokenize text
    message_tokenized = tokenize_single(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    words_lemmatized = [lemmatizer.lemmatize(token) for token in message_tokenized]
    
    # Apply porter stemmer
    ps = PorterStemmer()
    words_stemmed = [ps.stem(word) for word in words_lemmatized]
    
    return words_stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [138]:
from sklearn.pipeline import Pipeline
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import hamming_loss

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.metrics import classification_report

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV




In [139]:
Y.sum(axis=1)
categories_colnames = list(Y.columns)
categories_colnames

X_subset = X[:2]
Y_subset = Y.head(2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# X_train, X_test, Y_train, Y_test = train_test_split(X_subset, Y_subset)

# Y_train["request"]

print("Done.")

Done.


In [140]:
# print(classification_report(Y_test["related"], prediction))
# print(classification_report(Y_test["related"], model_dict["related"].predict(X_test)))



# Test with MultiOutputClassifier()

In [141]:
pipeline_modular = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        # ('clf', MultiOutputClassifier(RandomForestClassifier())),
        # ('clf', MultiOutputClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
        ('clf', MultiOutputClassifier(SGDClassifier())),
        # ('clf', MLPClassifier(random_state=1, max_iter=300)),



    ])

# train the model using X_dtm & y
pipeline_modular.fit(X_train, Y_train)
# compute the testing accuracy
prediction_modular = pipeline_modular.predict(X_test)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


In [142]:
print(classification_report(Y_test, prediction_modular, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_modular)}")

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4970
               request       0.79      0.55      0.65      1121
                 offer       0.00      0.00      0.00        35
           aid_related       0.77      0.68      0.72      2778
          medical_help       0.70      0.18      0.29       539
      medical_products       0.73      0.19      0.30       350
     search_and_rescue       0.74      0.12      0.20       198
              security       0.00      0.00      0.00       119
              military       0.69      0.12      0.20       211
                 water       0.74      0.61      0.67       415
                  food       0.81      0.73      0.77       749
               shelter       0.83      0.52      0.64       593
              clothing       0.68      0.50      0.58        86
                 money       0.64      0.06      0.12       142
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [143]:
print("Test")

Test


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [144]:
parameters = 1

cv = 1

In [145]:
["hinge", "log_loss", "log", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]

['hinge',
 'log_loss',
 'log',
 'modified_huber',
 'squared_hinge',
 'perceptron',
 'squared_error',
 'huber',
 'epsilon_insensitive',
 'squared_epsilon_insensitive']

In [146]:
parameters_algorithms = {
    'clf__estimator': [SGDClassifier(), MultinomialNB(), RandomForestClassifier(), KNeighborsClassifier()],
}

grid_search_model = GridSearchCV(pipeline_modular, param_grid=parameters_algorithms)

In [147]:
# train the model using X_dtm & y
grid_search_model.fit(X_train, Y_train)
# compute the testing accuracy
prediction_grid_search_model = grid_search_model.predict(X_test)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

<class 'Exception'>
Text '    ' replaced with empty string.


c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


In [ ]:
print(classification_report(Y_test, prediction_grid_search_model, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_grid_search_model)}")

pd.DataFrame(grid_search_model.cv_results_)

                        precision    recall  f1-score   support

               related       0.89      0.95      0.92        43
               request       0.81      0.91      0.86        33
                 offer       0.00      0.00      0.00         0
           aid_related       0.88      0.92      0.90        38
          medical_help       0.00      0.00      0.00         9
      medical_products       0.00      0.00      0.00         5
     search_and_rescue       0.00      0.00      0.00         3
              security       0.00      0.00      0.00         1
              military       0.00      0.00      0.00         0
                 water       1.00      0.78      0.88        18
                  food       0.90      0.53      0.67        17
               shelter       0.00      0.00      0.00         7
              clothing       0.00      0.00      0.00         1
                 money       0.00      0.00      0.00         2
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.570004,0.056339,0.056583,0.069507,SGDClassifier(),{'clf__estimator': SGDClassifier()},0.100000,NaN,NaN,0.033333,NaN,NaN,NaN,4
1,0.722552,0.204017,0.204860,0.093918,MultinomialNB(),{'clf__estimator': MultinomialNB()},0.000000,0.066667,0.066667,0.000000,0.033333,0.033333,0.029814,3
2,5.986384,0.335549,0.498399,0.088969,RandomForestClassifier(),{'clf__estimator': RandomForestClassifier()},0.066667,0.200000,0.200000,0.033333,0.133333,0.126667,0.067987,1
3,0.607298,0.044656,0.310610,0.045570,KNeighborsClassifier(),{'clf__estimator': KNeighborsClassifier()},0.066667,0.066667,0.133333,0.033333,0.133333,0.086667,0.040000,2


In [ ]:
stop_code

In [ ]:
# parameters = {
#     # 'clf__SGDClassifier__loss': ["hinge", "log_loss"]
#     'clf__estimator': [SGDClassifier(), MultinomialNB()],
#     # "clf__estimator__loss": ["hinge", "log_loss"]

# }

# grid_search_model = GridSearchCV(pipeline_modular, param_grid=parameters)

In [ ]:
# train the model using X_dtm & y
grid_search_model.fit(X_train, Y_train)
# compute the testing accuracy
prediction_grid_search = grid_search_model.predict(X_test)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token

In [ ]:
print(classification_report(Y_test, prediction_grid_search, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_grid_search)}")

                        precision    recall  f1-score   support

               related       0.89      0.92      0.91       211
               request       0.71      0.86      0.78       142
                 offer       0.00      0.00      0.00         1
           aid_related       0.77      0.77      0.77       161
          medical_help       0.46      0.23      0.31        26
      medical_products       0.54      0.41      0.47        17
     search_and_rescue       0.14      0.11      0.12         9
              security       0.00      0.00      0.00         7
              military       0.00      0.00      0.00         1
                 water       0.84      0.68      0.75        56
                  food       0.84      0.89      0.86        72
               shelter       0.62      0.60      0.61        30
              clothing       1.00      0.33      0.50         9
                 money       0.00      0.00      0.00         6
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [ ]:
pd.DataFrame(grid_search_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.917711,0.231106,0.691495,0.034281,SGDClassifier(),{'clf__estimator': SGDClassifier()},0.166667,0.18,0.186667,0.18,0.153333,0.173333,0.011926,1
1,3.015823,0.211967,0.743383,0.034666,MultinomialNB(),{'clf__estimator': MultinomialNB()},0.060000,0.04,0.060000,0.04,0.066667,0.053333,0.011155,2


In [ ]:
stop_code

NameError: name 'stop_code' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
filename_model = "ML_model.sav"

file_model = open(filename_model, "wb")
pickle.dump(pipeline_modular, file_model)
file_model.close()

In [ ]:
# import model
file_model = open(filename_model, "rb")
model_imported = pickle.load(file_model)
file_model.close()

In [ ]:
prediction_loaded = model_imported.predict(X_test)

print(classification_report(Y_test, prediction_loaded, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_loaded)}")

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4912
               request       0.78      0.57      0.66      1118
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.68      0.71      2688
          medical_help       0.66      0.20      0.31       516
      medical_products       0.72      0.22      0.34       338
     search_and_rescue       0.81      0.12      0.21       178
              security       0.00      0.00      0.00       122
              military       0.73      0.17      0.27       215
                 water       0.74      0.65      0.69       404
                  food       0.81      0.71      0.76       739
               shelter       0.82      0.50      0.62       614
              clothing       0.64      0.47      0.55        95
                 money       0.57      0.06      0.11       129
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

# Old Code Pieces

In [ ]:
stop_code  

In [ ]:

pipeline_OvR = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', OneVsRestClassifier(RandomForestClassifier())),
    ])

# train the model using X_dtm & y
pipeline_OvR.fit(X_train, Y_train)
# compute the testing accuracy
prediction_OvR = pipeline_OvR.predict(X_test)

print(classification_report(Y_test, prediction_OvR))

pipeline_NB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
    ])

# train the model using X_dtm & y
pipeline_NB.fit(X_train, Y_train)
# compute the testing accuracy
prediction_NB = pipeline_NB.predict(X_test)

print(classification_report(Y_test, prediction_NB, target_names=categories_colnames))
# SVM
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pipeline_SVM = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(SGDClassifier())),
    ])

# train the model using X_dtm & y
pipeline_SVM.fit(X_train, Y_train)
# compute the testing accuracy
prediction_SVM = pipeline_SVM.predict(X_test)

print(classification_report(Y_test, prediction_SVM))
hamming_loss(Y_test, prediction_SVM)
# Test Multilayer Perceptron MLP
from sklearn.neural_network import MLPClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MLPClassifier(random_state=1, max_iter=300)),
    ])

# train the model using X_dtm & y
pipeline.fit(X_train, Y_train)
# compute the testing accuracy
prediction = pipeline.predict(X_test)

In [ ]:
# # categories_colnames = ["related"] #, "request"]

# # https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize_single)),
# #         ('tfidf', TfidfVectorizer(lowercase=False)),
#         ('tfidf', TfidfTransformer()),

#         ('clf', OneVsRestClassifier(MultinomialNB(
#             fit_prior=True, class_prior=None))),
#     ])

# model_dict = {}

# for category in categories_colnames:
#     print(f'... Processing {category}')
#     # train the model using X_dtm & y
#     pipeline.fit(X_train, Y_train[category])
#     # compute the testing accuracy
#     prediction = pipeline.predict(X_test)
#     print(f'Test accuracy is {accuracy_score(Y_test[category], prediction)}')
#     model_dict[category] = pipeline